Basic Imports

In [1]:
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bs4 as bs
import ftplib
import gzip
import os
import pandas as pd
import psycopg2
import re
import sys
import time
import requests
import urllib.request
import pip
import zipfile

c:\Users\Jeremias Junior\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Auxiliary functions

In [2]:
def check_diff(url, file_name):
    '''
    Verifica se o arquivo no servidor existe no disco e se ele tem o mesmo
    tamanho no servidor.
    '''
    if not os.path.isfile(file_name):
        return True # ainda nao foi baixado

    response = requests.head(url)
    new_size = int(response.headers.get('content-length', 0))
    old_size = os.path.getsize(file_name)
    if new_size != old_size:
        os.remove(file_name)
        return True # tamanho diferentes

    return False # arquivos sao iguais

def makedirs(path):
    '''
    cria path caso seja necessario
    '''
    if not os.path.exists(path):
        os.makedirs(path)

def getEnv(env):
    return os.getenv(env)

Getting the directories for the data

In [3]:
local_env = r'C:\Users\Jeremias Junior\Documents\GitHub\gov_data'

dados_rf = 'http://200.152.38.155/CNPJ/'

raw_files = r'C:\Users\Jeremias Junior\Documents\GitHub\gov_data\data\raw_files'
extracted_files = r'C:\Users\Jeremias Junior\Documents\GitHub\gov_data\data\extracted_files'

raw_html = urllib.request.urlopen(dados_rf)
raw_html = raw_html.read()

# Formatar página e converter em string
page_items = bs.BeautifulSoup(raw_html, 'lxml')
html_str = str(page_items)

Files = []
text = '.zip'
for m in re.finditer(text, html_str):
    i_start = m.start()-40
    i_end = m.end()
    i_loc = html_str[i_start:i_end].find('href=')+6
    Files.append(html_str[i_start+i_loc:i_end])

Extracting files

In [ ]:
i_l = 0
print('unzipping files')
for l in Files:

    try:
        i_l += 1
        
        print(str(i_l) + ' - ' + l)
        full_path = os.path.join(raw_files, l)
        with zipfile.ZipFile(full_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_files)
    except:
        pass

Setting up files and database

In [4]:
insert_start = time.time()

items = [name for name in os.listdir(extracted_files) if name.endswith('')]

files = dict()

files['empresa'] = list()
files['estabelecimento'] = list()
files['socios'] = list()
files['simples'] = list()
files['cnae'] = list()
files['moti'] = list()
files['munic'] = list()
files['natju'] = list()
files['pais'] = list()
files['quals'] = list()

for i in range(len(items)):

    if items[i].find('EMPRE') > -1:
        files['empresa'].append(items[i])
    if items[i].find('ESTABELE') > -1:
        files['estabelecimento'].append(items[i])
    if items[i].find('SOCIO') > -1:
        files['socios'].append(items[i])
    if items[i].find('SIMPLES') > -1:
        files['simples'].append(items[i])
    if items[i].find('CNAE') > -1:
        files['cnae'].append(items[i])
    if items[i].find('MOTI') > -1:
        files['moti'].append(items[i])
    if items[i].find('MUNIC') > -1:
        files['munic'].append(items[i])
    if items[i].find('NATJU') > -1:
        files['natju'].append(items[i])
    if items[i].find('PAIS') > -1:
        files['pais'].append(items[i])
    if items[i].find('QUALS') > -1:   
        files['quals'].append(items[i])

    else:
        pass


import pyodbc
import sqlalchemy

               
conn = pyodbc.connect(driver='{SQL Server}', 
                      server='(local)', 
                      database='gov_db',               
                      trusted_connection='yes')

cursor = conn.cursor()



Empresa

In [5]:
table_name = 'empresa'
table_columns = ['cnpj_basico', 
                       'razao_social', 
                       'natureza_juridica', 
                       'qualificacao_responsavel', 
                       'capital_social', 
                       'porte_empresa', 
                       'ente_federativo_responsavel']

cursor.execute('DROP TABLE IF EXISTS "empresa";')
conn.commit()


create_table= f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in table_columns])})"
cursor.execute(create_table)
cursor.commit()


insertdata_query = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({', '.join(['?'] * len(table_columns))})"


for i in range(0, len(files['empresa'])):
    
    print('loading : ',files['empresa'][i])
    empresa = pd.DataFrame(columns=[0, 1, 2, 3, 4, 5, 6])
    empresa_dtypes = {0: 'object', 1: 'object', 2: 'object', 3: 'object', 4: 'object', 5: 'object', 6: 'object'}
    extracted_file_path = os.path.join(extracted_files, files['empresa'][i])

    empresa = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          nrows=1000,
                          skiprows=0,
                          header=None,
                          dtype=empresa_dtypes,
                          encoding='latin-1',
                        )
    
    empresa = empresa.reset_index()

    del empresa['index']

    empresa.columns = table_columns
    
    empresa['capital_social'] = empresa['capital_social'].apply(lambda x: x.replace(',','.'))
    empresa['capital_social'] = empresa['capital_social'] .astype(float)
    empresa['ente_federativo_responsavel'] = empresa['ente_federativo_responsavel'] .astype(str)
    
    for values in empresa.values.tolist():
        cursor.execute(insertdata_query, values)

conn.commit()

    
    

  




loading :  K3241.K03200Y1.D20910.EMPRECSV
loading :  K3241.K03200Y2.D20910.EMPRECSV
loading :  K3241.K03200Y3.D20910.EMPRECSV
loading :  K3241.K03200Y4.D20910.EMPRECSV
loading :  K3241.K03200Y5.D20910.EMPRECSV
loading :  K3241.K03200Y6.D20910.EMPRECSV
loading :  K3241.K03200Y7.D20910.EMPRECSV
loading :  K3241.K03200Y8.D20910.EMPRECSV
loading :  K3241.K03200Y9.D20910.EMPRECSV


Estabelecimento

In [6]:
table_name = 'estabelecimento'

table_columns = ['cnpj_basico',
                 'cnpj_ordem',
                 'cnpj_dv',
                 'identificador_matriz_filial',
                 'nome_fantasia',
                 'situacao_cadastral',
                               'data_situacao_cadastral',
                               'motivo_situacao_cadastral',
                               'nome_cidade_exterior',
                               'pais',
                               'data_inicio_atividade',
                               'cnae_fiscal_principal',
                               'cnae_fiscal_secundaria',
                               'tipo_logradouro',
                               'logradouro',
                               'numero',
                               'complemento',
                               'bairro',
                               'cep',
                               'uf',
                               'municipio',
                               'ddd_1',
                               'telefone_1',
                               'ddd_2',
                               'telefone_2',
                               'ddd_fax',
                               'fax',
                               'correio_eletronico',
                               'situacao_especial',
                               'data_situacao_especial']

insertdata_query = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({', '.join(['?'] * len(table_columns))})"
cursor.execute('DROP TABLE IF EXISTS "estabelecimento";')

create_table= f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(1024)' for col in table_columns])})"
cursor.execute(create_table)
cursor.commit()

for i in range(0, len(files['estabelecimento'])):
    
    print('loading : ',files['estabelecimento'][i])
    estabelecimento = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28])
    extracted_file_path = os.path.join(extracted_files, files['estabelecimento'][i])

    estabelecimento = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          nrows=1000,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
                        ).fillna('nan')

    estabelecimento = estabelecimento.reset_index()
    del estabelecimento['index']

    for values in estabelecimento.values.tolist():
        cursor.execute(insertdata_query, values)

conn.commit()


loading :  K3241.K03200Y0.D30610.ESTABELE
loading :  K3241.K03200Y1.D30610.ESTABELE
loading :  K3241.K03200Y2.D30610.ESTABELE
loading :  K3241.K03200Y3.D30610.ESTABELE
loading :  K3241.K03200Y4.D30610.ESTABELE
loading :  K3241.K03200Y5.D30610.ESTABELE
loading :  K3241.K03200Y6.D30610.ESTABELE
loading :  K3241.K03200Y7.D30610.ESTABELE
loading :  K3241.K03200Y8.D30610.ESTABELE
loading :  K3241.K03200Y9.D30610.ESTABELE


Socios

In [9]:
table_name = 'socios'
table_columns = ['cnpj_basico',
                      'identificador_socio',
                      'nome_socio_razao_social',
                      'cpf_cnpj_socio',
                      'qualificacao_socio',
                      'data_entrada_sociedade',
                      'pais',
                      'representante_legal',
                      'nome_do_representante',
                      'qualificacao_representante_legal',
                      'faixa_etaria']

insertdata_query = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({', '.join(['?'] * len(table_columns))})"
cursor.execute('DROP TABLE IF EXISTS "socios";')
conn.commit()

create_table= f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in table_columns])})"
cursor.execute(create_table)
cursor.commit()

for i in range(0, len(files['socios'])):

    print('loading : ',files['estabelecimento'][i])
    extracted_file_path = os.path.join(extracted_files, files['socios'][i])
    socios = pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10,11])
    socios = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          nrows=1000,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
                        ).fillna('nan')
    
    socios.columns = table_columns
    socios = socios.reset_index()
    del socios['index']

    for values in socios.values.tolist():
        print(values)
        cursor.execute(insertdata_query, values)
    

loading :  K3241.K03200Y0.D30610.ESTABELE
['46204351', '2', 'RUTHIELLI DOS SANTOS ORTIZ', '***627370**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '3']
['46204403', '2', 'LUIZ FELIPE FRANCA AMADIO', '***446578**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '4']
['46204403', '2', 'BRUNO DESIDERA RAPOSO', '***528398**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '4']
['46204413', '2', 'ANSELMO RICARDO DA SILVA', '***226797**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '5']
['46204456', '2', 'LAIS ROCHA ROBERTI', '***570166**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '3']
['46204480', '2', 'GIACOMO RAUPP ZANINI', '***226980**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '4']
['46204480', '2', 'RAFAELA FAVERO BANDEIRA', '***445880**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '4']
['46204513', '2', 'CARLOS FERNANDO DE CARLE RAMOS', '***429247**', '49', '20220429', 'nan', '***000000**', 'nan', '00', '7']
['46204557'

Simples Nacional


In [10]:
table_name = 'simples'
table_columns = ['cnpj_basico',
                           'opcao_pelo_simples',
                           'data_opcao_simples',
                           'data_exclusao_simples',
                           'opcao_mei',
                           'data_opcao_mei',
                           'data_exclusao_mei']

insertdata_query = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({', '.join(['?'] * len(table_columns))})"
cursor.execute('DROP TABLE IF EXISTS "simples";')
conn.commit()

create_table= f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in table_columns])})"
cursor.execute(create_table)
cursor.commit()

for i in range(0, len(files['simples'])):

    print('loading : ',files['simples'][i])
    extracted_file_path = os.path.join(extracted_files, files['simples'][i])
    simples = pd.DataFrame(columns=[1,2,3,4,5,6])
    simples = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          nrows=1000,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
                        ).fillna('nan')
    
    simples.columns = table_columns
    simples = simples.reset_index()
    del simples['index']

    for values in simples.values.tolist():
        print(values)
        cursor.execute(insertdata_query, values)
    

loading :  F.K03200$W.SIMPLES.CSV.D30610
['00000000', 'N', '20070701', '20070701', 'N', '20090701', '20090701']
['00000006', 'N', '20180101', '20191231', 'N', '00000000', '00000000']
['00000008', 'N', '20140101', '20211231', 'N', '00000000', '00000000']
['00000011', 'S', '20070701', '00000000', 'N', '00000000', '00000000']
['00000013', 'S', '20090101', '00000000', 'N', '00000000', '00000000']
['00000015', 'N', '20070701', '20081231', 'N', '00000000', '00000000']
['00000030', 'N', '20150101', '20151231', 'N', '00000000', '00000000']
['00000040', 'S', '20070701', '00000000', 'N', '00000000', '00000000']
['00000041', 'N', '20070701', '20151103', 'N', '00000000', '00000000']
['00000056', 'N', '20120101', '20141203', 'N', '00000000', '00000000']
['00000065', 'N', '20070701', '20191231', 'N', '00000000', '00000000']
['00000068', 'N', '20150101', '20151231', 'N', '00000000', '00000000']
['00000091', 'N', '20070701', '20071231', 'N', '00000000', '00000000']
['00000092', 'N', '20070701', '20080

cnae

In [ ]:
table_name = 'cnae'
table_columns = ['codigo', 'descricao']

insertdata_query = f"INSERT INTO {table_name} ({', '.join(table_columns)}) VALUES ({', '.join(['?'] * len(table_columns))})"
cursor.execute('DROP TABLE IF EXISTS "cnae";')
conn.commit()

create_table= f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in table_columns])})"
cursor.execute(create_table)
cursor.commit()

for i in range(0, len(files['cnae'])):

    print('loading : ',files['cnae'][i])
    extracted_file_path = os.path.join(extracted_files, files['cnae'][i])
    cnae = pd.DataFrame(columns=[1,2])
    cnae = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          nrows=1000,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
                        ).fillna('nan')
    
    cnae.columns = table_columns
    cnae = cnae.reset_index()
    del cnae['index']

    for values in cnae.values.tolist():
        print(values)
        cursor.execute(insertdata_query, values)
    